In [2]:
import urllib
import requests
import bs4

# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from fake_useragent import UserAgent

# 댓글 수집하기

In [3]:
url = 'http://corners.auction.co.kr/AllKill/AllDay.aspx'

## urllib을 이용한 html page 가져오기

In [4]:
file, http = urllib.request.urlretrieve(url, './data/auction_list.html')

- 이미지, 영상 등의 파일을 바로 저장이 가능

In [5]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [6]:
html = response.read()

## requests을 이용한 html page 가져오기

In [7]:
response = requests.get(url)

In [ ]:
# response.text
response.content

## 리눅스에서 selenium 설정

### chrome 설치
```
#> wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
#> sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
#> apt-get update
#> apt-get upgrade 
#> apt-get install google-chrome-stable
#> google-chrome --version
```

### 2. webdriver 다운로드
- url: https://chromedriver.chromium.org/downloads

```
#> wget https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_linux64.zip
#> unzip chromedriver_linux64.zip
```

## selenium을 통한 html page 가져오기

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('./data/chromedriver', options=options)

In [10]:
browser.get(url)
response = browser.page_source

browser.quit()

## BeautifulSoup을 이용한 html 파싱

In [11]:
# html 코드를 분석에 적당한 beautifulSoup 형태로 변환
html = bs4.BeautifulSoup( response, 'html.parser')

In [12]:
html.find('p')

<p class="layer-complainprocess__description"><strong>옥션은 거래 과정에서 발생하는 판매자와의 분쟁</strong> 또는 <strong>옥션이 제공하는 서비스에 대한 불만을 신속, 공정하게 해결</strong>하기 위해 고객님의 목소리에 귀를 기울이고 분쟁해결을 위해 최선을 다하겠습니다.</p>

In [13]:
condition = {
    'id':'coreSitemap',
    'class': 'core_sitemap'
}
html.find_all('div', attrs = condition)

[<div class="core_sitemap" id="coreSitemap" style="position: absolute; top: 0px; left: 40px; z-index: 1000002; display: none;"></div>]

- find()를 이용해서 원하는 속성의 조건을 검색할 수 있는 기능

In [14]:
element = html.select_one('ul#ItemList li a')

In [15]:
element['href']

'http://through.auction.co.kr/common/SafeRedirect.aspx?cc=178F&next=http%3a%2f%2fitempage3.auction.co.kr%2fDetailView.aspx%3fitemno%3dC222562959'

In [16]:
elements = html.select('ul#ItemList > li > div.inner > a')

In [17]:
item_list = []
for element in elements:
    if not 'javascript' in element['href']:
        # print(element['href'])
        item_list.append(element['href'])

- select()는 CSS SELECTOR를 통해서 원하는 요소를 검색할 수 있는 기능

## 댓글 가져오기

### urllib을 통해서 가져오기

In [18]:
for url in item_list:
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    data = response.read().decode('euc-kr')
    html = bs4.BeautifulSoup(data, 'html.parser')
    element = html.select('div#VIPReview div#sectionVipReview div#divVipReview')
    print(element)
    break

[<div id="divVipReview"></div>]


### requests를 통한 댓글 가져오기

In [19]:
for url in item_list:
    response = requests.get(url)
    html = bs4.BeautifulSoup(response.text, 'html.parser')
    element = html.select('div#VIPReview div#sectionVipReview div#divVipReview')
    print(element)
    break

[<div id="divVipReview"></div>]


### selenium을 통한 댓글 가져오기

In [20]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
for url in item_list:
    try:
        browser = webdriver.Chrome('./data/chromedriver', options=options)
        browser.get(url)
        response = browser.page_source

        html = bs4.BeautifulSoup(response, 'html.parser')
        elements = html.select('div#VIPReview div#sectionVipReview div#divVipReview p.text')
        # print(elements)
        for ele in elements:
            print(ele.get_text())
        browser.quit()
    except:
        browser.quit()
        break
    # break

# openAPI 이용하기

In [65]:
import os
import sys
import urllib.request
import json

client_id = "zgQm1ndA_GdvPFSL3Pfn"
client_secret = "yFBFM1DG3K"

encText = urllib.parse.quote("코로나")
news_list = []

for idx in range(0, 5):
    # url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + '&start=' + str(idx * 10 + 1)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        # print(response_body.decode('utf-8'))
        text = response_body.decode('utf-8')
        text = json.loads(text)
        news_list.append(text)
    else:
        print("Error Code:" + rescode)

In [47]:
len(news_list)

5

In [59]:
len(news_list[0]['items'])

10

In [63]:
news_list[0]['items'][0]['link']

'https://blog.naver.com/mc922js?Redirect=Log&logNo=222207292905'

In [68]:
urls = []
for pages in news_list:
    for item in pages['items']:
        if 'https://news.naver.com/main' in item['link']:
            # print(item['link'])
            urls.append(item['link'])

In [75]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [83]:
for url in urls:
    try:
        browser = webdriver.Chrome('./data/chromedriver', options=options)
        browser.get(url)
        response = browser.page_source
        soup = bs4.BeautifulSoup(response, 'html.parser')
        title = soup.find('h3', id='articleTitle').get_text()
        # print(title)
        
        body = soup.find('font', class_='article')
        print(body.get_text())
    except Exception as e:
        browser.quit()
        print(e)
        break
        
    browser.quit()
    break

김용범 기획재정부 1차관. 사진| 연합뉴스[스포츠서울 동효정 기자] 김용범 기획재정부 1차관이 설 민생안정대책으로 기부 릴레이 운동이 이어지기를 기대한다고 말했다. 30일 김 차관은 자신의 페이스북에 “기부를 통해 모인 자금은 위기 상황으로 인해 어려움을 겪는 저소득층의 생계비, 의료비, 교육비 지원에 사용될 예정”이라며 이같이 밝혔다.김 차관은 “명절을 계기로 열리는 기부 캠페인 행사로는 처음 시도되는 이번 프로그램은 코로나19 상황을 고려해 비대면 방식으로 진행된다”며 “QR 코드 및 각종 간편결제 방식을 활용해 누구나 손쉽게 기부에 참여할 수 있도록 설계됐다”고 설명했다.김 차관은 “정부는 기부문화 확산을 뒷받침하기 위해 올해 기부금에 대해서는 기부금 세액공제율을 한시 상향하는 방침도 확정했다”며 “구체적 추가공제율은 7월께 세법 개정안을 통해 발표할 예정”이라고 말했다.김 차관은 최근 한 신사가 추위에 떨며 커피 한잔 사달라는 서울역 노숙인에게 자신이 입고 있던 외투와 장갑, 현금 5만원을 건넨 사진 사례를 들며 “신사의 따뜻한 마음은 비단 노숙자뿐만 아니라 코로나19 장기화로 힘든 시기를 보내고 있는 국민들의마음까지 위로해줬다”고 했다. 기초생활보장 수급자 부부가 100만원을 기부한 사례도 소개했다. 김 차관은 “안타깝게도 올해는 지난해와 달리 비대면으로 마음만 전하는 설 명절이 될 가능성이 높아 보인다”며 “비록 사랑하는 가족들의 얼굴을 직접 보지는 못하지만 마음만은 어느 때보다 따뜻한 설 연휴가 되길 희망한다”고 말했다.  vivid@sportsseoul.com
